In [34]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.1.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:8 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:14 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:15 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic 

In [35]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2021-09-27 03:53:28--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar.1’

postgresql-42.2.9.j 100%[===================>] 892.61K  4.48MB/s    in 0.2s    

2021-09-27 03:53:29 (4.48 MB/s) - ‘postgresql-42.2.9.jar.1’ saved [914037/914037]



In [36]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [37]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Watches_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
watches_df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Watches_v1_00.tsv.gz"), sep="\t", header=True)
watches_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|    3653882|R3O9SGZBVQBV76|B00FALQ1ZC|     937001370|Invicta Women's 1...|         Watches|          5|            0|          0|   N|                Y|          Five Stars|Absolutely love t...| 2015-08-31|
|         US|   14661224| RKH8BNC3L5DLF|B00D3RGO20|     484010722|Kenneth Cole New ...|         Watches|          5|    

Schema - Watches_df

In [80]:
watches_df.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: string (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: string (nullable = true)
 |-- helpful_votes: string (nullable = true)
 |-- total_votes: string (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: string (nullable = true)



Count the number of records in the Database

In [38]:
# Row Count
watches_df.count()

960872

Transform the dataset to fit the tables in the schema file

In [39]:
from pyspark.sql.functions import to_date
# DataFrame
review_id = watches_df.select(["review_id", "customer_id", "product_id", "product_parent", to_date("review_date", 'yyyy-MM-dd').alias("review_date")])
review_id.show()

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R3O9SGZBVQBV76|    3653882|B00FALQ1ZC|     937001370| 2015-08-31|
| RKH8BNC3L5DLF|   14661224|B00D3RGO20|     484010722| 2015-08-31|
|R2HLE8WKZSU3NL|   27324930|B00DKYC7TK|     361166390| 2015-08-31|
|R31U3UH5AZ42LL|    7211452|B000EQS1JW|     958035625| 2015-08-31|
|R2SV659OUJ945Y|   12733322|B00A6GFD7S|     765328221| 2015-08-31|
| RA51CP8TR5A2L|    6576411|B00EYSOSE8|     230493695| 2015-08-31|
| RB2Q7DLDN6TH6|   11811565|B00WM0QA3M|     549298279| 2015-08-31|
|R2RHFJV0UYBK3Y|   49401598|B00A4EYBR0|     844009113| 2015-08-31|
|R2Z6JOQ94LFHEP|   45925069|B00MAMPGGE|     263720892| 2015-08-31|
| RX27XIIWY5JPB|   44751341|B004LBPB7Q|     124278407| 2015-08-31|
|R15C7QEZT0LGZN|    9962330|B00KGTVGKS|      28017857| 2015-08-31|
|R361XSS37V0NCZ|   16097204|B0039UT5OU|     685450910| 2015-08

In [43]:
from pyspark import SparkFiles
# DataFrame
products = watches_df.select(["product_id", "product_title"])
products.show() 

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B00FALQ1ZC|Invicta Women's 1...|
|B00D3RGO20|Kenneth Cole New ...|
|B00DKYC7TK|Ritche 22mm Black...|
|B000EQS1JW|Citizen Men's BM8...|
|B00A6GFD7S|Orient ER27009B M...|
|B00EYSOSE8|Casio Men's GW-94...|
|B00WM0QA3M|Fossil Women's ES...|
|B00A4EYBR0|INFANTRY Mens Nig...|
|B00MAMPGGE|G-Shock Men's Gre...|
|B004LBPB7Q|Heiden Quad Watch...|
|B00KGTVGKS|Fossil Women's ES...|
|B0039UT5OU|Casio General Men...|
|B00MPF0XJQ|2Tone Gold Silver...|
|B003P1OHHS|Bulova Men's 98B1...|
|B00R70YEOE|Casio - G-Shock -...|
|B000FVE3BG|Invicta Men's 332...|
|B008X6JB12|Seiko Women's SUT...|
|B0040UOFPW|Anne Klein Women'...|
|B00UR2R5UY|Guess U13630G1 Me...|
|B00HFF57L0|Nixon Men's Geo V...|
+----------+--------------------+
only showing top 20 rows



In [64]:
from pyspark import SparkFiles
# DataFrame
customers = watches_df.groupby(["customer_id"]).count()
customers = customers.withColumnRenamed("count", "customer_count")
customers.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   11234247|             1|
|   38676202|             1|
|   51245251|             1|
|   52523043|             1|
|   45717823|            13|
|   10828210|             6|
|   12854219|             1|
|   51715942|             1|
|   11477755|             2|
|   20024088|             3|
|    1486564|             1|
|    8648812|             1|
|    1619706|             1|
|   25622063|             2|
|   16435476|             8|
|    8062860|             1|
|    5632722|             2|
|   44537075|             1|
|   21072459|             1|
|   15367495|             1|
+-----------+--------------+
only showing top 20 rows



In [63]:
from pyspark import SparkFiles
vine_table = watches_df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine"])
vine_table.show()

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R3O9SGZBVQBV76|          5|            0|          0|   N|
| RKH8BNC3L5DLF|          5|            0|          0|   N|
|R2HLE8WKZSU3NL|          2|            1|          1|   N|
|R31U3UH5AZ42LL|          5|            0|          0|   N|
|R2SV659OUJ945Y|          4|            0|          0|   N|
| RA51CP8TR5A2L|          5|            0|          0|   N|
| RB2Q7DLDN6TH6|          5|            1|          1|   N|
|R2RHFJV0UYBK3Y|          1|            1|          5|   N|
|R2Z6JOQ94LFHEP|          5|            1|          2|   N|
| RX27XIIWY5JPB|          4|            0|          0|   N|
|R15C7QEZT0LGZN|          4|            2|          2|   N|
|R361XSS37V0NCZ|          1|            0|          0|   N|
| ROTNLALUAJAUB|          3|            0|          0|   N|
|R2DYX7QU6BGOHR|          5|            

Load the Dataframes that correspond to tables into an RDS instance

In [77]:
mode = 'append'
jdbc_url = "jdbc:postgresql://mypostgresdb.ceomfsf1omq9.us-east-2.rds.amazonaws.com:5432/my_data_class_db"
config = {
    'user':'postgres',

    'driver':"org.postgresql.Driver"
}



In [78]:
review_id.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

Py4JJavaError: ignored

In [ ]:
  products.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)